In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List

# ✅ Correct imports for latest LangGraph version
from langgraph.graph import StateGraph, START, END

from langchain_openai import ChatOpenAI
from tavily import TavilyClient

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# Define the data structure (state) that will flow through the graph
class GraphState(TypedDict):
    question: str
    documents: List[str]
    answer: str

# Define the nodes for our graph
def retrieve_documents(state: GraphState):
    # This node gets documents from the web based on the question
    print("--- Retrieving Documents ---")
    question = state.get('question', '')
    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    results = tavily.search(query=question, max_results=3)
    documents = [res['content'] for res in results['results']]
    return {"documents": documents}

def generate_answer(state: GraphState):
    # This node generates an answer using the retrieved documents
    print("--- Generating Answer ---")
    question = state.get('question', '')
    documents = state.get('documents', [])
    
    prompt = f"""You are an assistant for question-answering tasks.
    Use the following retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    
    Context: {documents}
    Question: {question}
    Answer:"""
    
    llm = ChatOpenAI(model="gpt-4o")
    response = llm.invoke(prompt)
    return {"answer": response.content}

# Build the graph
workflow = StateGraph(GraphState)

# Add the nodes
workflow.add_node("retriever", retrieve_documents)
workflow.add_node("generator", generate_answer)

# Define the edges to connect the nodes in sequence
workflow.add_edge(START, "retriever")
workflow.add_edge("retriever", "generator")
workflow.add_edge("generator", END)

# Compile the graph into a runnable application
app = workflow.compile()

# Run the graph and stream the results
inputs = {"question": "What are the key features of the James Webb Space Telescope?"}
for event in app.stream(inputs):
    for key, value in event.items():
        print(f"--- {key} ---")
        print(value)


--- Retrieving Documents ---
--- retriever ---
{'documents': ['The James Webb Space Telescope is the most powerful space telescope, using infrared light to observe distant objects, with a 6.5m mirror, and', 'As the largest telescope in space, it is equipped with high-resolution and high-sensitivity instruments, allowing it to view objects too old, distant, or faint', 'The James Webb Space Telescope is the most advanced telescope, focusing on infrared, can see distant galaxies, and can see back to the beginning of the']}
--- Generating Answer ---
--- generator ---
{'answer': 'The key features of the James Webb Space Telescope include its ability to use infrared light to observe distant objects, its large 6.5-meter mirror, and its high-resolution and high-sensitivity instruments. These features allow it to view objects that are too old, distant, or faint, and to see back to the beginning of the universe.'}
